In [28]:
from IPython.display import display
from typing import List,Dict
import os
import sys
import psycopg2
os.chdir('/home/rchuzh/programming/image_labelling_shrdc')
from pathlib import Path
Path.cwd()
SRC = Path('/home/rchuzh/programming/image_labelling_shrdc/src')  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

In [ ]:
from data_manager.database_manager import db_fetchone
dsn = "host=localhost port=5432 dbname=eye user=shrdc password=shrdc"
def init_connection(dsn=None, connection_factory=None, cursor_factory=None, **kwargs):
    """ Connect to the PostgreSQL database server """

    try:
        # read connection parameters
        # params = config()

        # connect to the PostgreSQL server
        display('Connecting to the PostgreSQL database...')
        if kwargs:
            conn = psycopg2.connect(**kwargs)

        else:
            conn = psycopg2.connect(dsn, connection_factory, cursor_factory)

        # create a cursor
        with conn:
            with conn.cursor() as cur:

                # execute a statement
                cur.execute('SELECT version();')
                conn.commit()

                # display the PostgreSQL database server version
                db_version = cur.fetchall()[0]
                display(f"PostgreSQL database version: {db_version}")
                display(f"PostgreSQL connection status: {conn.info.status}")
                display(
                    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        display(error)
        conn = None


In [ ]:
conn=init_connection(dsn)

# Editor Config

In [4]:
from xml.dom import minidom
import xml.dom

In [6]:
data="""
<View>
<Header value="Select label and start to click on image"/>
  <View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">
    <Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>
    <View>
      <Filter toName="tag" minlength="0" name="filter"/>
      <RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">
        <Label value="Hello" background="blue"/>
        <Label value="World" background="pink"/>
      </RectangleLabels>
    </View>
  </View>
</View>
"""

In [8]:
doc=minidom.parseString(data)
doc

In [9]:
doc.getElementsByTagName('Label')[0].attributes.items()

[('value', 'Hello'), ('background', 'blue')]

In [10]:
doc.childNodes

[<DOM Element: View at 0x7f81aa692310>]

## Create Node

In [11]:
def create_element(doc,parent,tagname,attr,value):
    nodeList = doc.getElementsByTagName(parent)[0]
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    new_label=doc.createElement(tagname)
    new_label.setAttribute(attr,value)
    
    nodeList.appendChild(new_label)

In [12]:
create_element(doc,'RectangleLabels','Label','value','Hello')

In [13]:
rectangle=doc.getElementsByTagName('RectangleLabels')[0]
print(rectangle)
new_label=doc.createElement('Label')
new_label.setAttribute("value","aruco")
new_label.setAttribute('background','red')
rectangle.appendChild(new_label)

<DOM Element: RectangleLabels at 0x7f81aadd1dc0>


<DOM Element: Label at 0x7f81aa692b80>

## Remove Nodes

In [15]:
def removeNodes(doc,tagname,attr,value):
    nodeList = doc.getElementsByTagName(tagname)
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    for node in reversed(nodeList):
        if node.hasAttribute(attr) and node.getAttribute(attr) == value:
            parent = node.parentNode
            parent.removeChild(node)

## Edit Nodes

In [16]:
def set_attribute(doc,tagname,attr,value,new_value):
    nodeList = doc.getElementsByTagName(tagname)
    # in php removing node has to be done in reverse order or it changes the
    #  nodeList. I assume same is true with python minidom
    for node in reversed(nodeList):
        if node.hasAttribute(attr) and node.getAttribute(attr) == value:
            node.setAttribute(attr,new_value)

In [17]:
set_attribute(doc,'Label','value','Hello','aruco')

In [18]:
removeNodes(doc,'Label','value','Hello')

## Print DOM

In [26]:
def xml_string(xml_doc):
    xml_string=xml_doc.toxml(encoding='utf8').decode('utf-8')
    return xml_string

In [25]:
print(xml_string(doc))

'<?xml version="1.0" encoding="utf8"?>\n<View>\n\t\n\n\t<Header value="Select label and start to click on image"/>\n\t\n  \n\t<View style="display:flex;align-items:start;gap:8px;flex-direction:column-reverse">\n\t\t\n    \n\t\t<Image name="img" value="$image" zoom="true" zoomControl="true" rotateControl="false" grid="true" brightnessControl="true" contrastControl="true"/>\n\t\t\n    \n\t\t<View>\n\t\t\t\n      \n\t\t\t<Filter toName="tag" minlength="0" name="filter"/>\n\t\t\t\n      \n\t\t\t<RectangleLabels name="tag" toName="img" showInline="true" fillOpacity="0.5" strokeWidth="5">\n\t\t\t\t\n        \n\t\t\t\t<Label value="aruco" background="blue"/>\n\t\t\t\t\n        \n\t\t\t\t<Label value="World" background="pink"/>\n\t\t\t\t\n      \n\t\t\t\t<Label value="aruco"/>\n\t\t\t\t<Label value="aruco" background="red"/>\n\t\t\t</RectangleLabels>\n\t\t\t\n    \n\t\t</View>\n\t\t\n  \n\t</View>\n\t\n\n</View>\n'